# **알약 데이터 증강 기법 상세 가이드 (Albumentations)**

---

## **I. 색상 관련 증강 기법**

알약은 색상이 종류를 구분하는 핵심적인 특징이므로, 색상 관련 증강 기법 적용 시 매우 신중해야 합니다.

### **1. 반드시 적용하는 것이 좋은 증강 기법**

알약의 본질적인 색상 정보를 해치지 않으면서, 실제 조명 환경 변화에 모델이 강건해지도록 돕습니다.

| 증강 기법 (명령어)             | 설명                                                                   | 적절한 수치 / 파라미터 (예시)                                              |
| :----------------------------- | :--------------------------------------------------------------------- | :------------------------------------------------------------------------- |
| **`A.RandomBrightnessContrast`** | 이미지의 밝기(Brightness)와 대비(Contrast)를 무작위로 조절             | `brightness_limit=(-0.2, 0.2)`, `contrast_limit=(-0.2, 0.2)`, `p=0.3`     |
| **`A.GaussNoise`**             | 이미지에 가우시안 노이즈(카메라 센서 노이즈)를 추가                  | `p=0.1`                                                                    |
| **`A.Blur`**                   | 이미지를 약하게 흐리게 만듦 (초점 불량 시뮬레이션)                 | `blur_limit=(3, 3)`, `p=0.1`                                               |
| **`A.MotionBlur`**             | 이미지에 움직임 블러 효과 추가 (움직임 촬영 시뮬레이션)            | `blur_limit=(3, 3)`, `p=0.1`                                               |
| **`A.RandomGamma`**            | 이미지의 감마 값을 조절하여 밝기 곡선을 비선형적으로 변경          | `gamma_limit=(80, 120)`, `p=0.2`                                           |
| **`A.HueSaturationValue`**     | 색조(Hue), 채도(Saturation), 명도(Value)를 미세하게 조절           | `hue_shift_limit=5`, `sat_shift_limit=10`, `val_shift_limit=20`, `p=0.3` |

### **2. 가급적 피하거나 절대 피해야 할 증강 기법**

| 증강 기법 (명령어)           | 이유                                                                         |
| :--------------------------- | :--------------------------------------------------------------------------- |
| **`A.ToGray` / `A.Grayscale`** | 알약의 색상 정보를 완전히 제거하여 핵심 식별 단서를 없앱니다.                  |
| **극단적인 `hue_shift_limit`** | 알약의 본연 색깔을 알아보지 못할 정도로 심하게 변형시킵니다.                    |
| **`blur_limit`가 매우 큰 `A.Blur`** | 알약 각인, 분할선 등 중요한 세부 특징을 손상시켜 식별을 어렵게 만듭니다.      |
| **랜덤 색상 채널 제거**      | 알약의 중요한 색상 정보를 제거하여 모델이 오판할 확률을 높입니다.            |

---

## **II. 기하학적 증강 기법**

알약의 형태, 각인, 그리고 다양한 배치 상황을 모방하기 위해 중요하지만, 지나친 변형은 식별을 어렵게 할 수 있습니다.

### **1. 반드시 적용하는 것이 좋은 증강 기법**

알약의 본질적인 형태와 각인 정보를 최대한 보존하면서, 실제 환경에서 발생할 수 있는 위치/방향/크기 변화에 모델이 대응하도록 합니다.

| 증강 기법 (명령어)             | 설명                                                                       | 적절한 수치 / 파라미터 (예시)                                                  |
| :----------------------------- | :------------------------------------------------------------------------- | :----------------------------------------------------------------------------- |
| **`A.HorizontalFlip`**         | 이미지를 수평(좌우)으로 반전                                               | `p=0.5`                                                                        |
| **`A.Rotate`**                 | 이미지를 0도부터 360도까지 임의의 각도로 회전 (일반적인 회전)              | `limit=30` (±30도), `p=0.3`, `border_mode=cv2.BORDER_CONSTANT` (배경은 검은색으로) |
| **`A.ShiftScaleRotate`** (이동) | 이미지를 수평 또는 수직으로 임의의 거리에 이동 (객체 위치 변화 시뮬레이션) | `shift_limit=0.05` (전체 이미지의 ±5%), `scale_limit=0`, `rotate_limit=0`, `p=0.3` |

### **2. 신중하게 적용하고 강도 조절이 필수인 증강 기법**

알약의 각인과 형태를 왜곡할 수 있으므로, 반드시 낮은 강도로 시작하여 성능을 면밀히 관찰해야 합니다.

| 증강 기법 (명령어)              | 설명                                                               | 적절한 수치 / 파라미터 (예시)                                                       |
| :------------------------------ | :----------------------------------------------------------------- | :---------------------------------------------------------------------------------- |
| **`A.ShiftScaleRotate`** (확대/축소) | 이미지 내 알약 객체의 크기를 임의로 확대하거나 축소              | `scale_limit=(0.8, 1.2)` (원본 대비 80%~120%), `p=0.3`                            |
| **`A.Perspective`**             | 이미지를 비스듬하게 기울이거나 원근감을 주어 변형               | `scale=(0.05, 0.1)` (작은 왜곡), `p=0.1`, `pad_val=0`                               |
| **`A.ShiftScaleRotate`** (전단) | 이미지를 전단(Shear) 변형 (이미지를 한쪽으로 밀어 기울이는 효과) | `shear_limit=5` (최대 ±5도), `p=0.1`                                               |
| **`A.PiecewiseAffine`**         | 이미지를 여러 부분으로 나누어 미세하게 변형 (알약 표면의 굴곡 시뮬레이션) | `scale=(0.01, 0.03)` (매우 약한 스케일), `nb_rows=4`, `nb_cols=4`, `p=0.05` |

### **3. 가급적 피하거나 절대 피해야 할 증강 기법**

| 증강 기법 (명령어)             | 이유                                                                                         |
| :----------------------------- | :------------------------------------------------------------------------------------------- |
| **`A.VerticalFlip`**           | 대부분의 알약은 각인이 있어 상하 대칭이 아닙니다. 각인 정보가 뒤집혀 오인식될 확률이 높습니다. |
| **극단적인 형태 왜곡**         | 알약의 크기나 형태를 비현실적으로 변형시켜 모델 학습에 혼란을 줍니다.                   |
| **`CoarseDropout` / `A.Cutout` 과도한 적용** | 바운딩 박스 내부의 알약 객체 전체를 가려버려 '라벨은 있는데 객체는 안 보이는' 상황을 만들어 성능을 저하시킵니다. |

---

## **III. 예시 증강 파이프라인 (Albumentations `A.Compose` 함수)**

아래는 위에서 설명된 기법들을 조합하여 구현한 `Albumentations` 기반의 파이프라인 예시입니다. 이들을 `ny_augmentation.py` 파일에 추가하여 테스트해볼 수 있습니다.

```python
import albumentations as A
import cv2 
from albumentations.pytorch import ToTensorV2

# TARGET_IMAGE_SIZE는 프로젝트 전반에 걸쳐 사용되는 이미지 크기를 정의
# 예: (640, 640) 또는 모델 입력에 맞게 조절
TARGET_IMAGE_SIZE = (640, 640) 

# --- 필수 및 보수적인 증강 파이프라인 ---
# 알약의 핵심 특징을 최대한 보존하면서 강건성을 확보하는, 가장 기본적인 파이프라인.
# [적용된 주요 증강] 좌우반전, 약한 회전, 약한 이동, 밝기/대비, 약한 노이즈/블러, 미세 색조/채도.
def get_train_transforms_conservative(target_size: tuple = TARGET_IMAGE_SIZE) -> A.Compose:
    return A.Compose([
        # --- 기하학적 변형 ---
        A.HorizontalFlip(p=0.5), # 좌우 반전
        A.Rotate(limit=30, p=0.3, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0)), # ±30도 범위 회전 (각인 보존)
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0, rotate_limit=0, p=0.3, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0)), # 약한 이동 (shift_limit=0.05), 스케일/회전은 0으로 설정하여 중복 방지

        # --- 색상 및 노이즈 변형 ---
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3), # 밝기/대비
        A.GaussNoise(p=0.1), # 약한 가우시안 노이즈
        A.Blur(blur_limit=3, p=0.1), # 약한 블러
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=20, p=0.3), # 채도/색조 (매우 낮은 강도)
        A.RandomGamma(p=0.2), # 감마 조절

        # --- 고정 전처리 (순서 중요) ---
        A.LongestMaxSize(max_size=max(target_size), p=1.0), # 가장 긴 축을 target_size에 맞게 조절
        A.PadIfNeeded( # target_size에 맞게 패딩 추가
            min_height=target_size[1],
            min_width=target_size[0],
            border_mode=cv2.BORDER_CONSTANT,
            value=(0,0,0), # 검은색으로 패딩
            p=1.0
        ),
        A.Normalize( # 표준화 (YOLO 기본 값 사용)
            mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0, p=1.0
        ),
        ToTensorV2(p=1.0) # PyTorch 텐서로 변환
    ], 
    bbox_params=A.BboxParams( # 바운딩 박스 변환 설정
        format='yolo', 
        label_fields=['class_labels'],
        min_area=1.0, min_visibility=0.0, clip=True, min_width=1, min_height=1
    ))

# --- 균형 잡힌 증강 파이프라인 ---
# Conservative 파이프라인에 신중하게 적용할 기법들과 가려짐/압축 손실 시뮬레이션 기법들을 추가하여 성능을 더 끌어올립니다.
# [적용된 주요 증강] Conservative 기법 + 확대/축소, 컷아웃, JPEG 압축
def get_train_transforms_balanced(target_size: tuple = TARGET_IMAGE_SIZE) -> A.Compose:
    return A.Compose([
        # --- 기하학적 변형 ---
        A.HorizontalFlip(p=0.5), # 좌우 반전
        A.Rotate(limit=60, p=0.4, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0)), # ±60도 범위 회전 (Conservative보다 확장)
        A.ShiftScaleRotate(shift_limit=0.075, scale_limit=0.15, rotate_limit=0, p=0.4, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0)), # 약한 이동 (shift_limit=0.075), 확대/축소 (scale_limit=0.15)
        
        # --- 색상 및 노이즈 변형 ---
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
        A.GaussNoise(p=0.1),
        A.Blur(blur_limit=3, p=0.1),
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=20, p=0.3),
        A.RandomGamma(p=0.2),
        
        # --- 가려짐 및 파일 손상 시뮬레이션 ---
        A.CoarseDropout(max_holes=1, max_height=0.1, max_width=0.1, min_holes=1, fill_value=0, p=0.1), # 컷아웃 (부분 가려짐)
        A.JpegCompression(quality_lower=70, quality_upper=90, p=0.1), # JPEG 압축 손실

        # --- 고정 전처리 (순서 중요) ---
        A.LongestMaxSize(max_size=max(target_size), p=1.0), 
        A.PadIfNeeded(
            min_height=target_size[1], min_width=target_size[0], border_mode=cv2.BORDER_CONSTANT, value=(0,0,0), p=1.0
        ),
        A.Normalize(
            mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0, p=1.0
        ),
        ToTensorV2(p=1.0)
    ], 
    bbox_params=A.BboxParams(
        format='yolo', 
        label_fields=['class_labels'],
        min_area=1.0, min_visibility=0.0, clip=True, min_width=1, min_height=1
    ))

# --- 공격적인/실험적인 증강 파이프라인 ---
# Balanced 파이프라인에 더욱 다양한 기법 (원근, 전단, 부분 왜곡 등)을 추가하여 잠재적 성능 향상을 탐색합니다.
# 강도가 높으므로 오버피팅 또는 성능 저하 가능성도 있으니 주의 깊은 모니터링이 필요합니다.
# [적용된 주요 증강] Balanced 기법 + 원근 변환, 전단, 부분 왜곡, 증강 강도/확률 전반적 상향
def get_train_transforms_aggressive(target_size: tuple = TARGET_IMAGE_SIZE) -> A.Compose:
    return A.Compose([
        # --- 기하학적 변형 ---
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=90, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0)), # ±90도까지 회전 범위 확장
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=0, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=(0,0,0), # 이동/확대/축소 강도 및 확률 증가
                           interpolation=cv2.INTER_LINEAR), # 보간법 지정
        A.Perspective(scale=(0.075, 0.15), p=0.2, pad_val=0), # 원근 변환 추가 (강도 및 확률 증가)
        A.PiecewiseAffine(scale=(0.01, 0.03), nb_rows=4, nb_cols=4, p=0.1), # 부분 왜곡 추가 (매우 약하게 시작)
        A.VerticalFlip(p=0.1), # 신중한 상하 반전 (각인 중요하지 않다면)
        
        # --- 색상 및 노이즈 변형 ---
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.4), # 밝기/대비 강도 및 확률 증가
        A.GaussNoise(p=0.15), # 노이즈 확률 증가
        A.Blur(blur_limit=5, p=0.15), # 블러 강도 및 확률 증가
        A.MotionBlur(blur_limit=5, p=0.15),
        A.HueSaturationValue(hue_shift_limit=7, sat_shift_limit=15, val_shift_limit=25, p=0.4), # 채도/색조 강도 및 확률 약간 증가
        A.RandomGamma(p=0.3),
        
        # --- 가려짐 및 파일 손상 시뮬레이션 ---
        A.CoarseDropout(max_holes=2, max_height=0.15, max_width=0.15, min_holes=1, fill_value=0, p=0.15), # 컷아웃 강도/확률 증가
        A.JpegCompression(quality_lower=60, quality_upper=85, p=0.15), # JPEG 압축 강도/확률 증가

        # --- 고정 전처리 (순서 중요) ---
        A.LongestMaxSize(max_size=max(target_size), p=1.0),
        A.PadIfNeeded(
            min_height=target_size[1], min_width=target_size[0], border_mode=cv2.BORDER_CONSTANT, value=(0,0,0), p=1.0
        ),
        A.Normalize(
            mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0, p=1.0
        ),
        ToTensorV2(p=1.0)
    ], 
    bbox_params=A.BboxParams(
        format='yolo', 
        label_fields=['class_labels'],
        min_area=1.0, min_visibility=0.0, clip=True, min_width=1, min_height=1
    ))